In [102]:
'''
Here we develop our own scraper for funda.nl. 
This will help us get enough data on Dutch properties to perform some real estate analysis.
'''
#first we find the data elements for a single house using selenium as normal html scraping doesn't work
from selenium import webdriver
import time

PATH = "/snap/bin/chromium.chromedriver"
driver = webdriver.Chrome(PATH)
source = driver.get(url="https://www.funda.nl/en/koop/amsterdam/huis-42623862-bosplaat-8/")
# print("Element identified by clazss:",driver.find_element_by_class_name("object-kenmerken-body").text)
print("Views", driver.find_element_by_class_name("object-statistics"))
# time.sleep(5)
# driver.close()

/tmp/ipykernel_9296/1661029030.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


Views <selenium.webdriver.remote.webelement.WebElement (session="80be988f8d359c03bf27a0e4f04d1c2e", element="dbb70ca3-138e-4307-aad0-7f3d5c2c2e95")>


/tmp/ipykernel_9296/1661029030.py:13: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  print("Views", driver.find_element_by_class_name("object-statistics"))


In [103]:
#all the feature information
print("Element identified by class:",driver.find_element_by_class_name("object-kenmerken-body").text)

/tmp/ipykernel_9296/1606485465.py:2: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  print("Element identified by class:",driver.find_element_by_class_name("object-kenmerken-body").text)


Element identified by class: Transfer of ownership
Asking price
€ 1,150,000 kosten koper
Asking price per m²
€ 5,637
Listed since
Today
Status
Available
Acceptance
Available in consultation
Construction
Kind of house
Single-family home, corner house (patio residence)
Building type
Resale property
Year of construction
1975
Type of roof
Flat roof
Surface areas and volume
Areas
Living area
204 m²
Other space inside the building
4 m²
External storage space
17 m²
Plot size
229 m²
Volume in cubic meters
811 m³
Layout
Number of rooms
6 rooms (4 bedrooms)
Number of bath rooms
2 bathrooms and 1 separate toilet
Bathroom facilities
Bath, shower and 2 toilets
Number of stories
4 stories
Facilities
Alarm installation, outdoor awning, optical fibre, mechanical ventilation, passive ventilation system, flue, sliding door, TV via cable and solar panels
Energy
Energy label
B What does this mean?
Insulation
Double glazing and energy efficient window
Heating
CH boiler
Hot water
CH boiler
CH boiler
ATAG (g

In [80]:
#real estate broker
print("Real estate broker:", driver.find_element_by_class_name("object-contact-aanbieder-link").text)

Views Ankersmit Makelaars Amsterdam


/tmp/ipykernel_9296/2641911625.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  print("Views", driver.find_element_by_class_name("object-contact-aanbieder-link").text)


In [84]:
#get neighbourhoud
print(driver.find_element_by_class_name("object-buurt").text)

Views Neighborhood
Buikslotermeer
Amsterdam
More about the neighborhood
In the neighborhood
Elementary school
Supermarket
Bus stop
Discover locations in the neighborhood


/tmp/ipykernel_9296/659161760.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  print("Views", driver.find_element_by_class_name("object-buurt").text)


In [99]:
list = driver.find_elements_by_class_name("object-statistics")
for item in list:
    print(item.text)
# /html/body/main/div[2]/div/div[4]/div/div/div/div/ul/li[1]/div[2]
# document.querySelector("#content > div.object-detail.fd-container--xl.fd-m-auto > div > div.object-buurt > div > div > div > div > ul > li:nth-child(1) > div.text-right.md\\:text-left.rightCardColumn.text-dark-1")
# //*[@id="content"]/div[2]/div/div[4]/div/div/div/div/ul/li[1]/div[2]

/tmp/ipykernel_9296/3186512266.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  list = driver.find_elements_by_class_name("object-statistics")


In [ ]:
#once we managed to get data for a single house we need to find a way to get all the links to all the houses in a search
#so we can iterate all the house links and create a dataframe

In [121]:
import re
import pandas as pd
#all urls in the first search page for Amsterdam


links = set()

driver = webdriver.Chrome(PATH)
driver.get(url="https://www.funda.nl/en/koop/amsterdam/")
elems = driver.find_elements_by_xpath("//a[@href]")
for elem in elems:
    link = elem.get_attribute("href")
    if re.search('amsterdam/huis-', link) or re.search('amsterdam/appartement-', link):
        links.add(link)
        print(link)
        
print(links)
            
    
time.sleep(5)
driver.close()

/tmp/ipykernel_9296/3231495550.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
/tmp/ipykernel_9296/3231495550.py:10: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  elems = driver.find_elements_by_xpath("//a[@href]")


https://www.funda.nl/en/koop/amsterdam/appartement-88076813-pieter-van-der-doesstraat-34-b/?navigateSource=toppositie
https://www.funda.nl/en/koop/amsterdam/huis-42623862-bosplaat-8/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42623862-bosplaat-8/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42623862-bosplaat-8/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42623869-bosplaat-8-pp/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42623869-bosplaat-8-pp/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42623869-bosplaat-8-pp/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-88085850-gooise-kant-77/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-88085850-gooise-kant-77/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-88085850-gooise-kant-77/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/hui

In [115]:
for search in range(2,5):
    driver = webdriver.Chrome(PATH)
    driver.get(url="https://www.funda.nl/en/koop/amsterdam/"+"p"+str(search))
    elems = driver.find_elements_by_xpath("//a[@href]")
    for elem in elems:
        link = elem.get_attribute("href")
        if re.search('amsterdam/huis-', link):
            links.add(link)
            print(link)
    time.sleep(2)
    driver.close()
    

print(links)

/tmp/ipykernel_9296/2463633044.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
/tmp/ipykernel_9296/2463633044.py:4: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  elems = driver.find_elements_by_xpath("//a[@href]")


https://www.funda.nl/en/koop/amsterdam/huis-42613232-zwenkgrasstraat-5/?navigateSource=toppositie
https://www.funda.nl/en/koop/amsterdam/huis-42616862-morrahof-6/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42616862-morrahof-6/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42616862-morrahof-6/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42615279-jan-olphert-vaillantlaan-52/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42615279-jan-olphert-vaillantlaan-52/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42615279-jan-olphert-vaillantlaan-52/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42615279-jan-olphert-vaillantlaan-52/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42615279-jan-olphert-vaillantlaan-52/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/huis-42615279-jan-olphert-vaillantlaan-52/?navigat

In [117]:
len(links)

26

In [133]:
links = set()
last_page = 128


driver = webdriver.Chrome(PATH)
driver.get(url="https://www.funda.nl/en/koop/amsterdam/")
elems = driver.find_elements(by=By.XPATH, value="//a[@href]")
for elem in elems:
    link = elem.get_attribute("href")
    if re.search('amsterdam/huis-', link) or re.search('amsterdam/appartement-', link):
        links.add(link)
print(len(links))
time.sleep(2)
driver.close()

for page in range(2, last_page):
    driver = webdriver.Chrome(PATH)
    driver.get(url="https://www.funda.nl/en/koop/amsterdam/p"+str(page))
    elems = driver.find_elements(by=By.XPATH, value="//a[@href]")
    for elem in elems:
        link = elem.get_attribute("href")
        if re.search('amsterdam/huis-', link) or re.search('amsterdam/appartement-', link):
            links.add(link)
    print(len(links))
    time.sleep(2)
    driver.close()
    

print(links)
with open('links.txt','w') as f:
   f.write(str(links))  # set of numbers & a tuple

/tmp/ipykernel_9296/2118149970.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


18


/tmp/ipykernel_9296/2118149970.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


36
52
68
83
98
113
129
145
162
177
195
210
228
244
260
275
293
309
325
342
358
373
388
409
425
440
455
495
514
529
544
560
575
591
606
621
637
652
668
682
698
713
727
742
758
772
787
802
816
832
847
863
878
893
907
922
937
952
967
982
997
1013
1027
1042
1057
1071
1086
1101
1116
1131
1146
1160
1174
1188
1204
1219
1236
1250
1261
1276
1294
1309
1320
1329
1343
1353
1361
1376
1385
1393
1410
1425
1440
1460
1479
1493
1508
1523
1537
1552
1566
1581
1595
1611
1625
1641
1655
1671
1686
1701
1716
1730
1745
1760
1775
1796
1811
1829
1851
1869
1894
1913
1927
1948
1972
1983
{'https://www.funda.nl/en/koop/amsterdam/huis-42681504-oudezijds-voorburgwal-153-a/?navigateSource=resultlist', 'https://www.funda.nl/en/koop/amsterdam/appartement-41385605-amstelveenseweg-615-b/?navigateSource=resultlist', 'https://www.funda.nl/en/koop/amsterdam/appartement-42611304-kaap-york-6066-bouwnr-66/', 'https://www.funda.nl/en/koop/amsterdam/appartement-88028098-rustenburgerstraat-372-1/?navigateSource=resultlist', 'https:/

In [134]:
driver = webdriver.Chrome(PATH)
source = driver.get(url="https://www.funda.nl/en/koop/amsterdam/huis-42623862-bosplaat-8/")
# print("Element identified by clazss:",driver.find_element_by_class_name("object-kenmerken-body").text)
print("Views", driver.find_element_by_class_name("object-statistics"))
# time.sleep(5)
# driver.close()
#all the feature information
print("Element identified by class:",driver.find_element_by_class_name("object-kenmerken-body").text)

/tmp/ipykernel_9296/1716297239.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


Views <selenium.webdriver.remote.webelement.WebElement (session="c0f0e76d16db90d7c0b007b85228be94", element="55e5bdf9-d512-4d3d-9c22-12e5d27b9fe0")>
Element identified by class: Transfer of ownership
Asking price
€ 1,150,000 kosten koper
Asking price per m²
€ 5,637
Listed since
March 8, 2022
Status
Available
Acceptance
Available in consultation
Construction
Kind of house
Single-family home, corner house (patio residence)
Building type
Resale property
Year of construction
1975
Type of roof
Flat roof
Surface areas and volume
Areas
Living area
204 m²
Other space inside the building
4 m²
External storage space
17 m²
Plot size
229 m²
Volume in cubic meters
811 m³
Layout
Number of rooms
6 rooms (4 bedrooms)
Number of bath rooms
2 bathrooms and 1 separate toilet
Bathroom facilities
Bath, shower and 2 toilets
Number of stories
4 stories
Facilities
Alarm installation, outdoor awning, optical fibre, mechanical ventilation, passive ventilation system, flue, sliding door, TV via cable and solar pa

/tmp/ipykernel_9296/1716297239.py:4: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  print("Views", driver.find_element_by_class_name("object-statistics"))
/tmp/ipykernel_9296/1716297239.py:8: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  print("Element identified by class:",driver.find_element_by_class_name("object-kenmerken-body").text)


In [225]:
source = driver.get(url="https://www.funda.nl/en/koop/amsterdam/huis-42681504-oudezijds-voorburgwal-153-a/?navigateSource=resultlist")
string = driver.find_element_by_class_name("object-kenmerken-body").text
without_header = driver.find_elements_by_class_name("object-kenmerken-list")

data = []
for items in without_header:
    string = items.text
    string = string.split('\n')
    data.extend(string)
    
data.remove('Areas')
print(data)

/tmp/ipykernel_9296/2228685424.py:2: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  string = driver.find_element_by_class_name("object-kenmerken-body").text
/tmp/ipykernel_9296/2228685424.py:3: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  without_header = driver.find_elements_by_class_name("object-kenmerken-list")


['Asking price', '€ 1,740,000 kosten koper', 'Asking price per m²', '€ 6,374', 'Listed since', '6 weeks', 'Status', 'Available', 'Acceptance', 'Available in consultation', 'Kind of house', 'Single-family home, row house', 'Building type', 'Resale property', 'Year of construction', '1773', 'Specific', 'Protected townscape or village view (permit needed for alterations)', 'Type of roof', 'Gable roof', 'Living area', '273 m²', 'Exterior space attached to the building', '10 m²', 'Plot size', '158 m²', 'Volume in cubic meters', '1,001 m³', 'Number of rooms', '5 rooms (4 bedrooms)', 'Number of bath rooms', '2 bathrooms', 'Bathroom facilities', 'Bath, 2 showers and 2 toilets', 'Number of stories', '4 stories', 'Facilities', 'Passive ventilation system and TV via cable', 'Energy label', 'F What does this mean?', 'Insulation', 'Roof insulation', 'Heating', 'Gas heaters', 'Hot water', 'Electrical boiler', 'AMSTERDAM G 7467', 'Cadastral map', 'Area', '103 m²', 'Ownership situation', 'Full ownersh

In [226]:
index_start = data.index('Cadastral map')-1
index_end = data.index('Location')
temp_end = data[index_end:len(data)]
data = data[0:index_start]
data.extend(temp_end)
#no domain expertise on cadastral data so remove
col = data[::2]
row = data[1::2]
print(col)
print(row)

['Asking price', 'Asking price per m²', 'Listed since', 'Status', 'Acceptance', 'Kind of house', 'Building type', 'Year of construction', 'Specific', 'Type of roof', 'Living area', 'Exterior space attached to the building', 'Plot size', 'Volume in cubic meters', 'Number of rooms', 'Number of bath rooms', 'Bathroom facilities', 'Number of stories', 'Facilities', 'Energy label', 'Insulation', 'Heating', 'Hot water', 'Location', 'Garden', 'Patio/atrium', 'Type of parking facilities']
['€ 1,740,000 kosten koper', '€ 6,374', '6 weeks', 'Available', 'Available in consultation', 'Single-family home, row house', 'Resale property', '1773', 'Protected townscape or village view (permit needed for alterations)', 'Gable roof', '273 m²', '10 m²', '158 m²', '1,001 m³', '5 rooms (4 bedrooms)', '2 bathrooms', 'Bath, 2 showers and 2 toilets', '4 stories', 'Passive ventilation system and TV via cable', 'F What does this mean?', 'Roof insulation', 'Gas heaters', 'Electrical boiler', 'Alongside busy road, 

In [174]:
res = dict(zip(col, row))
print(res)

{'Asking price': '€ 625,000 kosten koper', 'Asking price per m²': '€ 6,793', 'Service charges': '€ 195 per month', 'Listed since': '3 weeks', 'Status': 'Sold under reservation', 'Acceptance': 'Available in consultation', 'Type apartment': 'Upstairs apartment (apartment)', 'Building type': 'Resale property', 'Year of construction': '2021', 'Living area': '92 m²', 'Exterior space attached to the building': '12 m²', 'External storage space': '8 m²', 'Volume in cubic meters': '310 m³', 'Number of rooms': '3 rooms (2 bedrooms)', 'Number of bath rooms': '1 bathroom and 1 separate toilet', 'Bathroom facilities': 'Bath and toilet', 'Number of stories': '1 story', 'Located at': '3rd floor', 'Facilities': 'Elevator, mechanical ventilation, sliding door, TV via cable and solar panels', 'Energy label': 'A++ What does this mean?', 'Insulation': 'Completely insulated', 'Heating': 'Complete floor heating and heat pump', 'Hot water': 'Central facility', 'Cadastral map': 'AMSTERDAM A 8382', 'Ownership 

In [180]:
df = pd.DataFrame()
df = df.append(res, ignore_index=True)
df.head()

,Acceptance,Annual meeting,Asking price,Asking price per m²,Balcony/roof garden,Bathroom facilities,Building insurance,Building type,Cadastral map,Energy label,...,Registration with KvK,Reserve fund present,Service charges,Shed / storage,Status,Type apartment,Type of garage,Type of parking facilities,Volume in cubic meters,Year of construction
0,Available in consultation,No,"€ 625,000 kosten koper","€ 6,793",Balcony present and french balcony present,Bath and toilet,Yes,Resale property,AMSTERDAM A 8382,A++ What does this mean?,...,Yes,No,€ 195 per month,Storage box,Sold under reservation,Upstairs apartment (apartment),Parking place,Paid parking,310 m³,2021


In [219]:
text_file = open('links.txt', 'r')
lines = text_file.readline()
links = lines.split(', ')
# print(data)
for link in links:
    index = links.index(link)
    temp = link[1:len(link)-1]
    links[index] = temp
    
print(links[0])

https://www.funda.nl/en/koop/amsterdam/huis-42681504-oudezijds-voorburgwal-153-a/?navigateSource=resultlist


In [233]:
df = pd.DataFrame()

small = links[0:5]

for link in small:
    print(link)

    source = driver.get(url=link)
    without_header = driver.find_elements_by_class_name("object-kenmerken-list")

    data = []
    for items in without_header:
        string = items.text
        string = string.split('\n')
        data.extend(string)

    if 'Areas' in data:
        data.remove('Areas')
    #no domain expertise on cadastral data so remove
    if 'Cadastral map' in data:
        index_start = data.index('Cadastral map')-1
        index_end = data.index('Location')
        temp_end = data[index_end:len(data)]
        data = data[0:index_start]
        data.extend(temp_end)
    col = data[::2]
    row = data[1::2]
    res = dict(zip(col, row))
    df = df.append(res, ignore_index=True)
    time.sleep(3)
    
df.head()

https://www.funda.nl/en/koop/amsterdam/huis-42681504-oudezijds-voorburgwal-153-a/?navigateSource=resultlist


/tmp/ipykernel_9296/1008727963.py:9: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  without_header = driver.find_elements_by_class_name("object-kenmerken-list")


https://www.funda.nl/en/koop/amsterdam/appartement-41385605-amstelveenseweg-615-b/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/appartement-42611304-kaap-york-6066-bouwnr-66/
https://www.funda.nl/en/koop/amsterdam/appartement-88028098-rustenburgerstraat-372-1/?navigateSource=resultlist
https://www.funda.nl/en/koop/amsterdam/appartement-42557195-van-suchtelen-van-de-haarestraat-121/?navigateSource=resultlist


,Acceptance,Asking price,Asking price per m²,Bathroom facilities,Building type,Energy label,Exterior space attached to the building,Facilities,Garden,Heating,...,Nee,"Op eigen terrein, openbaar parkeren en parkeervergunningen",Soort appartement,Soort bouw,TV kabel,Verkoopdatum,Volle eigendom,Vraagprijs per m²,Service charges,Sun terrace
0,Available in consultation,"€ 1,740,000 kosten koper","€ 6,374","Bath, 2 showers and 2 toilets",Resale property,F What does this mean?,10 m²,Passive ventilation system and TV via cable,Patio/atrium,Gas heaters,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Available in consultation,"€ 550,000 kosten koper","€ 5,140",Bath and shower,Resale property,NaN,12 m²,Elevator and mechanical ventilation,NaN,CH boiler,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Available in consultation,"€ 1,500,000 vrij op naam","€ 9,317",NaN,New property,NaN,110 m²,Electricity,NaN,Complete floor heating and heat recovery unit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Opstalverzekering,Inschrijving KvK,Tussenverdieping (appartement),Bestaande bouw,Energielabel,1 maart 2022,Ligging,€ 7.346,NaN,NaN
4,NaN,"€ 545,000 vrij op naam","€ 6,124",Shower,New property,A+++ What does this mean?,22 m²,NaN,Sun terrace,District heating,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€ 133 per month,"19 m² (2,6 meter deep and 7,2 meter broad)"
